# Ensemble Learning

## Initial Imports

In [21]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [23]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [24]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [33]:
# Create our features
X = df.drop(columns="loan_status")

X = pd.get_dummies(X)

# Create our target
y = df["loan_status"]

In [34]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,total_il_high_credit_limit,home_ownership_le,verification_status_le,issue_d_le,pymnt_plan_le,initial_list_status_le,next_pymnt_d_le,application_type_le,hardship_flag_le,debt_settlement_flag_le
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,5.572240e+04,1.812779,0.669994,0.805542,0.0,0.876121,0.616839,0.139660,0.0,0.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,5.095845e+04,0.941313,0.719105,0.714932,0.0,0.329446,0.486161,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,1.270000e+02,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,2.288000e+04,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,4.200000e+04,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,0.000000,0.0,0.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,7.249900e+04,3.000000,1.000000,1.000000,0.0,1.000000,1.000000,0.000000,0.0,0.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.426964e+06,3.000000,2.000000,2.000000,0.0,1.000000,1.000000,1.000000,0.0,0.0


In [35]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [39]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split # had to google this. lol

X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=1, 
                                                   stratify=y)
X_train.shape


(51612, 85)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [40]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()


In [41]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
data_scaler.fit(X_train)

StandardScaler()

In [42]:
# Scale the training data
scaled_train = data_scaler.transform(X_train)
scaled_train

array([[-0.64978133, -0.30327574, -0.52669276, ..., -0.40497582,
         0.        ,  0.        ],
       [ 0.42378352, -0.42616892,  0.71269346, ..., -0.40497582,
         0.        ,  0.        ],
       [ 1.79013878, -0.82400957,  2.19335392, ..., -0.40497582,
         0.        ,  0.        ],
       ...,
       [ 0.81417074, -0.36368086,  1.17925683, ..., -0.40497582,
         0.        ,  0.        ],
       [-0.25939411, -0.42616892, -0.08220905, ..., -0.40497582,
         0.        ,  0.        ],
       [-0.16179731, -1.196855  , -0.05849257, ..., -0.40497582,
         0.        ,  0.        ]])

In [43]:
# Scale the test data
scaled_test = data_scaler.transform(X_test)
scaled_test

array([[-1.23536216, -1.196855  , -1.24170452, ...,  2.46928324,
         0.        ,  0.        ],
       [ 0.22858991,  0.06540381, -0.16370179, ...,  2.46928324,
         0.        ,  0.        ],
       [-0.45458772,  0.35493183, -0.68932999, ..., -0.40497582,
         0.        ,  0.        ],
       ...,
       [-1.13776535,  0.35493183, -1.07283345, ..., -0.40497582,
         0.        ,  0.        ],
       [ 1.30215476, -0.95315156,  0.45599132, ..., -0.40497582,
         0.        ,  0.        ],
       [-0.16179731,  0.46949327,  0.13942335, ..., -0.40497582,
         0.        ,  0.        ]])

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [48]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

model_balanced = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
model_balanced.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [67]:
# Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = model_balanced.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7624430424114966

In [68]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[   58,    29],
       [ 2427, 14691]], dtype=int64)

In [69]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.67      0.86      0.05      0.76      0.56        87
   low_risk       1.00      0.86      0.67      0.92      0.76      0.58     17118

avg / total       0.99      0.86      0.67      0.92      0.76      0.58     17205



In [70]:
# List the features sorted in descending order by feature importance
importances = model_balanced.feature_importances_
importances_sorted = sorted(zip(model_balanced.feature_importances_, X.columns), reverse=True)
importances_sorted


[(0.0816556546415653, 'total_rec_prncp'),
 (0.06898166416165401, 'last_pymnt_amnt'),
 (0.05908272477868344, 'total_rec_int'),
 (0.05096929339053498, 'total_pymnt_inv'),
 (0.04966983721100822, 'total_pymnt'),
 (0.032009718601528916, 'int_rate'),
 (0.021260542100764727, 'mths_since_recent_inq'),
 (0.01980369928041582, 'installment'),
 (0.01971432580363021, 'max_bal_bc'),
 (0.018419341479927766, 'dti'),
 (0.018276333924126455, 'issue_d_le'),
 (0.01801242568874898, 'bc_util'),
 (0.01797005283295479, 'out_prncp_inv'),
 (0.017698932341569357, 'revol_bal'),
 (0.01621293588651029, 'il_util'),
 (0.015786090881254867, 'out_prncp'),
 (0.01562311569567227, 'mo_sin_old_rev_tl_op'),
 (0.015289061008861092, 'avg_cur_bal'),
 (0.014966110090416474, 'all_util'),
 (0.01489841479222795, 'mths_since_recent_bc'),
 (0.014891710071449564, 'annual_inc'),
 (0.014152491389977364, 'total_bal_ex_mort'),
 (0.013870297126126039, 'bc_open_to_buy'),
 (0.013554167047615164, 'mths_since_rcnt_il'),
 (0.01278961221635773,

### Easy Ensemble Classifier

In [57]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
easy = EasyEnsembleClassifier(n_estimators = 100,random_state=1)
easy = easy.fit(X_train, y_train)

In [73]:
# Calculated the balanced accuracy score
y_pred = model_balanced.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7624430424114966

In [74]:
# Display the confusion matrix
y_pred = model_balanced.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   58,    29],
       [ 2427, 14691]], dtype=int64)

In [75]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.67      0.86      0.05      0.76      0.56        87
   low_risk       1.00      0.86      0.67      0.92      0.76      0.58     17118

avg / total       0.99      0.86      0.67      0.92      0.76      0.58     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    both had the same score

2. Which model had the best recall score?

    both had the same score

3. Which model had the best geometric mean score?

    both had the same score

4. What are the top three features?
for the total are teh pre, f1 & rec.